# Join
Copyright (c) Microsoft Corporation. All rights reserved.<br>
Licensed under the MIT License.<br>

In Data Prep you can easily join two Dataflows.

In [1]:
import azureml.dataprep as dprep

First, get the left side of the data into a shape that is ready for the join.

In [2]:
# get the first Dataflow and derive desired key column
dflow_left = dprep.read_csv(path='https://dpreptestfiles.blob.core.windows.net/testfiles/BostonWeather.csv')
dflow_left = dflow_left.derive_column_by_example(source_columns='DATE', new_column_name='date_timerange',
                                                 example_data=[('11/11/2015 0:54', 'Nov 11, 2015 | 12AM-2AM'),
                                                              ('2/1/2015 0:54', 'Feb 1, 2015 | 12AM-2AM'),
                                                              ('1/29/2015 20:54', 'Jan 29, 2015 | 8PM-10PM')])
dflow_left = dflow_left.drop_columns(['DATE'])

# convert types and summarize data
dflow_left = dflow_left.set_column_types(type_conversions={'HOURLYDRYBULBTEMPF': dprep.TypeConverter(dprep.FieldType.DECIMAL)})
dflow_left = dflow_left.filter(expression=~dflow_left['HOURLYDRYBULBTEMPF'].is_error())
dflow_left = dflow_left.summarize(group_by_columns=['date_timerange'],summary_columns=[dprep.SummaryColumnsValue('HOURLYDRYBULBTEMPF', dprep.api.engineapi.typedefinitions.SummaryFunction.MEAN, 'HOURLYDRYBULBTEMPF_Mean')] )

# cache the result so the steps above are not executed every time we pull on the data
import os
from pathlib import Path
cache_dir = str(Path(os.getcwd(), 'dataflow-cache'))
dflow_left.cache(directory_path=cache_dir)
dflow_left.head(5)

date_timerange  HOURLYDRYBULBTEMPF_Mean
0  Jan 1, 2015 | 12AM-2AM                22.000000
1   Jan 1, 2015 | 2AM-4AM                23.000000
2   Jan 1, 2015 | 4AM-6AM                23.000000
3   Jan 1, 2015 | 6AM-8AM                22.666667
4  Jan 1, 2015 | 8AM-10AM                26.500000

Now let's prepare the data for the right side of the join.

In [3]:
# get the second Dataflow and desired key column
dflow_right = dprep.read_csv(path='https://dpreptestfiles.blob.core.windows.net/bike-share/*-hubway-tripdata.csv')
dflow_right = dflow_right.keep_columns(['starttime', 'start station id'])
dflow_right = dflow_right.derive_column_by_example(source_columns='starttime', new_column_name='l_date_timerange',
                                                 example_data=[('2015-01-01 00:21:44', 'Jan 1, 2015 | 12AM-2AM')])
dflow_right = dflow_right.drop_columns('starttime')

# cache the results
dflow_right.cache(directory_path=cache_dir)
dflow_right.head(5)

l_date_timerange start station id
0  Jan 1, 2015 | 12AM-2AM              115
1  Jan 1, 2015 | 12AM-2AM               80
2  Jan 1, 2015 | 12AM-2AM               91
3  Jan 1, 2015 | 12AM-2AM              115
4  Jan 1, 2015 | 12AM-2AM              105

There are three ways you can join two Dataflows in Data Prep:
1. Create a `JoinBuilder` object for interactive join configuration.
2. Call ```join()``` on one of the Dataflows and pass in the other along with all other arguments.
3. Call ```Dataflow.join()``` method and pass in two Dataflows along with all other arguments.

We will explore the builder object as it simplifies the determination of correct arguments. 

In [4]:
# construct a builder for joining dataflow_l with dataflow_r
join_builder = dflow_left.builders.join(right_dataflow=dflow_right, left_column_prefix='l', right_column_prefix='r')

join_builder

JoinBuilder:
    join_key_pairs: None
    left_column_prefix: l
    right_column_prefix: r
    left_non_prefixed_columns: []
    right_non_prefixed_columns: []
    is_join_suggestion_applied: False
    is_suggested_join_key_generated: N/A

So far the builder has no properties set except default values.
From here you can set each of the options and preview its effect on the join result or use Data Prep to determine some of them.

Let's start with determining appropriate column prefixes for left and right side of the join and lists of columns that would not conflict and therefore don't need to be prefixed.

In [5]:
join_builder.detect_column_info()
join_builder

JoinBuilder:
    join_key_pairs: None
    left_column_prefix: l2_
    right_column_prefix: r_
    left_non_prefixed_columns: ['date_timerange', 'HOURLYDRYBULBTEMPF_Mean', 'KEY_generated']
    right_non_prefixed_columns: ['l_date_timerange', 'start station id', 'KEY_generated']
    is_join_suggestion_applied: False
    is_suggested_join_key_generated: N/A

You can see that Data Prep has performed a pull on both Dataflows to determine the column names in them. Given that `dataflow_r` already had a column starting with `l_` new prefix got generated which would not collide with any column names that are already present.
Additionally columns in each Dataflow that won't conflict during join would remain unprefixed.
This apprach to column naming is crucial for join robustness to schema changes in the data. Let's say that at some time in future the data consumed by left Dataflow will also have `l_date_timerange` column in it.
Configured as above the join will still run as expected and the new column will be prefixed with `l2_` ensuring that ig column `l_date_timerange` was consumed by some other future transformation it remains unaffected.

Note: `KEY_generated` is appended to both lists and is reserved for Data Prep use in case Autojoin is performed.

### Autojoin
Autojoin is a Data prep feature that determines suitable join arguments given data on both sides. In some cases Autojoin can even derive a key column from a number of available columns in the data.
Here is how you can use Autojoin:

In [6]:
# generate join suggestions
join_builder.generate_suggested_join()

# list generated suggestions
join_builder.list_join_suggestions()

'[{0: "\\nSuggestion:\\n    Left:\\n        Needs transform: False\\n        % of matched rows: 0.18604651162790697\\n    Right:\\n        Needs transform: False\\n        % of matched rows: 1.0\\n    Join keys: [(\'date_timerange\', \'l_date_timerange\')]\\n\\n"}]'

Now let's select the first suggestion and preview the result of the join.

In [7]:
# apply first suggestion
join_builder.apply_suggestion(0)

join_builder.preview(10)

date_timerange  HOURLYDRYBULBTEMPF_Mean         l_date_timerange  \
0   Jan 1, 2015 | 8AM-10AM                26.500000   Jan 1, 2015 | 8AM-10AM   
1   Jan 1, 2015 | 8AM-10AM                26.500000   Jan 1, 2015 | 8AM-10AM   
2   Jan 1, 2015 | 8AM-10AM                26.500000   Jan 1, 2015 | 8AM-10AM   
3   Jan 1, 2015 | 8AM-10AM                26.500000   Jan 1, 2015 | 8AM-10AM   
4   Jan 1, 2015 | 8AM-10AM                26.500000   Jan 1, 2015 | 8AM-10AM   
5  Jan 1, 2015 | 10AM-12PM                29.666667  Jan 1, 2015 | 10AM-12PM   
6  Jan 1, 2015 | 10AM-12PM                29.666667  Jan 1, 2015 | 10AM-12PM   
7  Jan 1, 2015 | 10AM-12PM                29.666667  Jan 1, 2015 | 10AM-12PM   
8  Jan 1, 2015 | 10AM-12PM                29.666667  Jan 1, 2015 | 10AM-12PM   
9  Jan 1, 2015 | 10AM-12PM                29.666667  Jan 1, 2015 | 10AM-12PM   

  start station id  
0               75  
1              115  
2               88  
3               90  
4              116  
5               88  
6               95  
7              116  
8              116  
9              110

Now, get our new joined Dataflow.

In [8]:
dflow_autojoined = join_builder.to_dataflow().drop_columns(['l_date_timerange'])

### Joining two Dataflows without pulling the data

If you don't want to pull on data and know what join should look like, you can always use the join method on the Dataflow.

In [9]:
dflow_joined = dprep.Dataflow.join(left_dataflow=dflow_left,
                                      right_dataflow=dflow_right,
                                      join_key_pairs=[('date_timerange', 'l_date_timerange')],
                                      left_column_prefix='l2_',
                                      right_column_prefix='r_')


In [10]:
dflow_joined.head(5)

l2_date_timerange  l2_HOURLYDRYBULBTEMPF_Mean      r_l_date_timerange  \
0  Jan 1, 2015 | 12AM-2AM                        22.0  Jan 1, 2015 | 12AM-2AM   
1  Jan 1, 2015 | 12AM-2AM                        22.0  Jan 1, 2015 | 12AM-2AM   
2  Jan 1, 2015 | 12AM-2AM                        22.0  Jan 1, 2015 | 12AM-2AM   
3  Jan 1, 2015 | 12AM-2AM                        22.0  Jan 1, 2015 | 12AM-2AM   
4  Jan 1, 2015 | 12AM-2AM                        22.0  Jan 1, 2015 | 12AM-2AM   

  r_start station id  
0                115  
1                 80  
2                 91  
3                115  
4                105

In [11]:
dflow_joined = dflow_joined.filter(expression=dflow_joined['r_start station id'] == '67')
df = dflow_joined.to_pandas_dataframe()
df

l2_date_timerange  l2_HOURLYDRYBULBTEMPF_Mean  \
0     Jan 1, 2015 | 8AM-10AM                   26.500000   
1     Jan 1, 2015 | 12PM-2PM                   32.333333   
2      Jan 1, 2015 | 2PM-4PM                   32.000000   
3      Jan 1, 2015 | 4PM-6PM                   30.666667   
4      Jan 1, 2015 | 4PM-6PM                   30.666667   
5      Jan 1, 2015 | 4PM-6PM                   30.666667   
6     Jan 1, 2015 | 8PM-10PM                   30.500000   
7     Jan 2, 2015 | 12PM-2PM                   40.000000   
8      Jan 2, 2015 | 2PM-4PM                   38.500000   
9      Jan 2, 2015 | 2PM-4PM                   38.500000   
10     Jan 2, 2015 | 2PM-4PM                   38.500000   
11     Jan 2, 2015 | 4PM-6PM                   36.333333   
12     Jan 2, 2015 | 4PM-6PM                   36.333333   
13     Jan 2, 2015 | 4PM-6PM                   36.333333   
14     Jan 2, 2015 | 4PM-6PM                   36.333333   
15     Jan 2, 2015 | 6PM-8PM                   34.000000   
16     Jan 2, 2015 | 6PM-8PM                   34.000000   
17     Jan 2, 2015 | 6PM-8PM                   34.000000   
18     Jan 2, 2015 | 6PM-8PM                   34.000000   
19     Jan 3, 2015 | 6AM-8AM                   23.000000   
20   Jan 29, 2015 | 8AM-10AM                   20.000000   
21   Jan 29, 2015 | 8AM-10AM                   20.000000   
22   Jan 29, 2015 | 8AM-10AM                   20.000000   
23   Jan 29, 2015 | 8AM-10AM                   20.000000   
24   Jan 29, 2015 | 12PM-2PM                   30.666667   
25   Jan 29, 2015 | 12PM-2PM                   30.666667   
26    Jan 29, 2015 | 2PM-4PM                   29.500000   
27    Jan 29, 2015 | 2PM-4PM                   29.500000   
28    Jan 29, 2015 | 2PM-4PM                   29.500000   
29    Jan 29, 2015 | 6PM-8PM                   28.666667   
30    Jan 29, 2015 | 6PM-8PM                   28.666667   
31    Jan 29, 2015 | 6PM-8PM                   28.666667   
32    Jan 29, 2015 | 6PM-8PM                   28.666667   
33    Jan 29, 2015 | 6PM-8PM                   28.666667   
34   Jan 29, 2015 | 8PM-10PM                   28.000000   
35   Jan 29, 2015 | 8PM-10PM                   28.000000   
36  Jan 29, 2015 | 10PM-12AM                   28.000000   
37  Jan 29, 2015 | 10PM-12AM                   28.000000   
38  Jan 29, 2015 | 10PM-12AM                   28.000000   

          r_l_date_timerange r_start station id  
0     Jan 1, 2015 | 8AM-10AM                 67  
1     Jan 1, 2015 | 12PM-2PM                 67  
2      Jan 1, 2015 | 2PM-4PM                 67  
3      Jan 1, 2015 | 4PM-6PM                 67  
4      Jan 1, 2015 | 4PM-6PM                 67  
5      Jan 1, 2015 | 4PM-6PM                 67  
6     Jan 1, 2015 | 8PM-10PM                 67  
7     Jan 2, 2015 | 12PM-2PM                 67  
8      Jan 2, 2015 | 2PM-4PM                 67  
9      Jan 2, 2015 | 2PM-4PM                 67  
10     Jan 2, 2015 | 2PM-4PM                 67  
11     Jan 2, 2015 | 4PM-6PM                 67  
12     Jan 2, 2015 | 4PM-6PM                 67  
13     Jan 2, 2015 | 4PM-6PM                 67  
14     Jan 2, 2015 | 4PM-6PM                 67  
15     Jan 2, 2015 | 6PM-8PM                 67  
16     Jan 2, 2015 | 6PM-8PM                 67  
17     Jan 2, 2015 | 6PM-8PM                 67  
18     Jan 2, 2015 | 6PM-8PM                 67  
19     Jan 3, 2015 | 6AM-8AM                 67  
20   Jan 29, 2015 | 8AM-10AM                 67  
21   Jan 29, 2015 | 8AM-10AM                 67  
22   Jan 29, 2015 | 8AM-10AM                 67  
23   Jan 29, 2015 | 8AM-10AM                 67  
24   Jan 29, 2015 | 12PM-2PM                 67  
25   Jan 29, 2015 | 12PM-2PM                 67  
26    Jan 29, 2015 | 2PM-4PM                 67  
27    Jan 29, 2015 | 2PM-4PM                 67  
28    Jan 29, 2015 | 2PM-4PM                 67  
29    Jan 29, 2015 | 6PM-8PM                 67  
30    Jan 29, 2015 | 6PM-8PM                 67  
31    Jan 